In [1]:
import pandas as pd
import pyodbc
import numpy as np

In [2]:
data = pd.read_csv (r'T:\Tableau\tableauJourneyToWork\Datasources\ModeShare_ALL_Years.csv')   
df = pd.DataFrame(data)

print(df)

     Year             Geography                              Mode  Estimate  \
0    2009                Coburg                           Bicycle         2   
1    2009                Coburg    Car, truck, or van - Carpooled        83   
2    2009                Coburg  Car, truck, or van - Drove alone       373   
3    2009                Coburg                        Motorcycle         0   
4    2009                Coburg                       Other means         3   
..    ...                   ...                               ...       ...   
580  2021  Salem Urbanized Area                        Motorcycle       212   
581  2021  Salem Urbanized Area                           Bicycle      1248   
582  2021  Salem Urbanized Area                            Walked      3067   
583  2021  Salem Urbanized Area                       Other means       782   
584  2021  Salem Urbanized Area                    Worked at home     11010   

     MOE_Est     Share  MOE_Share   SharePct  MOE_S

In [12]:
df.dtypes

Year              int64
Geography        object
Mode             object
Estimate          int64
MOE_Est           int64
Share           float64
MOE_Share       float64
SharePct        float64
MOE_SharePct    float64
dtype: object

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [5]:
df.Mode.unique()

array(['Bicycle', 'Car, truck, or van - Carpooled',
       'Car, truck, or van - Drove alone', 'Motorcycle', 'Other means',
       'Public transportation (excluding taxicab)', 'Taxicab', 'Walked',
       'Worked at home'], dtype=object)

In [15]:
df.replace({np.inf: np.nan, -np.inf: np.nan}, inplace=True)

In [16]:
df = df.fillna(0)

In [19]:
cursor.execute('''
            DROP TABLE mode_share;
            CREATE TABLE mode_share (
            year int,
            geo varchar(50),
            mode varchar(50),
            estimate int,
            moe_est int,
            share float,
            moe_share float,
            share_pct float,
            moe_share_pct float
            )
               ''')

In [20]:
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO mode_share (year, geo, mode, estimate, moe_est, share, moe_share, share_pct, moe_share_pct)
                VALUES (?,?,?,?,?,?,?,?,?)
                ''',
                   row.Year,
                   row.Geography,
                   row.Mode,
                   row.Estimate,
                   row.MOE_Est,
                   row.Share,
                   row.MOE_Share,
                   row.SharePct,
                   row.MOE_SharePct
                  )
conn.commit()